# Analisi di mercato sulle opportunità SAM archiviate

Questo notebook realizza una market intelligence sul database `sam_archived_opportunities_filtered.sqlite`,
utilizzando le funzioni presenti in `scripts/sam_market_analysis.py`. L'obiettivo è estrarre insight
operativi per il management su volumi, valori economici, stakeholder istituzionali, programmi di set aside e
pattern temporali o geografici delle opportunità federali archiviate.


In [1]:
from pathlib import Path
import sys

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
SCRIPTS_PATH = PROJECT_ROOT / "scripts"
if str(SCRIPTS_PATH) not in sys.path:
    sys.path.insert(0, str(SCRIPTS_PATH))

from scripts.sam_market_analysis import (
    get_connection,
    list_archived_tables,
    load_opportunities,
    enrich_dataset,
    yearly_summary,
    agency_mix,
    naics_opportunity_matrix,
    set_aside_landscape,
    geographic_distribution,
    timeline_by_quarter,
    opportunity_duration_profile,
    awardee_leaderboard,
    award_concentration,
    award_amount_summary,
)

pd.set_option("display.float_format", lambda v: f"{v:,.2f}")
DB_PATH = Path("../db/sam_archived_opportunities_filtered.sqlite")


## Comprensione del perimetro dati
Partiamo dalla lista di tabelle fiscali disponibili e dalla loro dimensione per calibrare le analisi successive.


In [2]:
with get_connection(DB_PATH) as conn:
    tables_info = pd.DataFrame([info.__dict__ for info in list_archived_tables(conn)])

tables_info_sorted = tables_info.sort_values("fiscal_year").reset_index(drop=True)
display(tables_info_sorted)
print(f"Totale opportunità censite: {int(tables_info_sorted['rows'].sum()):,}")


,table,fiscal_year,rows
0,fy1970_archived_opportunities,1970,0
1,fy1980_archived_opportunities,1980,0
2,fy1998_archived_opportunities,1998,0
3,fy1999_archived_opportunities,1999,0
4,fy2000_archived_opportunities,2000,0
5,fy2001_archived_opportunities,2001,0
6,fy2002_archived_opportunities,2002,0
7,fy2003_archived_opportunities,2003,0
8,fy2004_archived_opportunities,2004,227
9,fy2005_archived_opportunities,2005,375


Totale opportunità censite: 16,610


## Caricamento e preparazione del dataset unificato
Uniamo tutte le tabelle disponibili, arricchendo il dataset con colonne normalizzate utili all'analisi (date coerenti,
importi numerici, durata delle finestre di risposta, standardizzazione dei campi testuali).


In [3]:
raw_df = load_opportunities(DB_PATH)
df = enrich_dataset(raw_df)

print(f"Shape finale: {df.shape[0]:,} righe × {df.shape[1]} colonne")
subset_cols = [
    "FiscalYear",
    "NoticeId",
    "Title",
    "Department/Ind.Agency",
    "Sub-Tier",
    "Type",
    "BaseType",
    "SetASide",
    "NaicsCode",
    "ClassificationCode",
    "AwardAmount",
    "State",
    "PostedDate",
    "ResponseDeadline",
    "ResponseWindowDays",
]
display(df[subset_cols].head())


Shape finale: 16,610 righe × 51 colonne


,FiscalYear,NoticeId,Title,Department/Ind.Agency,Sub-Tier,Type,BaseType,SetASide,NaicsCode,ClassificationCode,AwardAmount,State,PostedDate,ResponseDeadline,ResponseWindowDays
0,2004,db4ee0d982312e75c6f6c8d4c0d07ccf,R -- Screening of Passengers and Property at A...,"HOMELAND SECURITY, DEPARTMENT OF",TRANSPORTATION SECURITY ADMINISTRATION,Special Notice,Special Notice,,561612,R,NaN,VA,2004-09-29 05:00:00,NaT,NaN
1,2004,c85dc90a87b7700a778e6e855cb46ee6,S -- Armed Guard Security Services and Canine ...,,,Presolicitation,Presolicitation,8a Competed,561612,S,NaN,,2004-09-29 05:00:00,NaT,NaN
2,2004,627ca8b08ac95a4a2c679ae757e917bc,R -- Pre-solicitation meeting for the aquisiti...,,,Special Notice,Special Notice,,561612,R,NaN,,2004-09-28 05:00:00,NaT,NaN
3,2004,c6b379820584e280b584626e25afd941,R -- A-76 PUBLIC ANNOUNCEMENT,,,Presolicitation,Presolicitation,,561612,R,NaN,,2004-09-24 05:00:00,2004-10-25 05:00:00,31.00
4,2004,6b4b2148c2afd437b2492bfcb1c77c55,R -- A-76 PUBLIC ANNOUNCEMENT,,,Presolicitation,Presolicitation,,561612,R,NaN,,2004-09-24 05:00:00,2004-10-25 05:00:00,31.00


### Qualità dei principali campi numerici
Valutiamo la presenza di valori mancanti sulle variabili economiche e operative più rilevanti.


In [4]:
key_columns = [
    "AwardAmount",
    "ResponseWindowDays",
    "SetASide",
    "NaicsCode",
    "ClassificationCode",
]
missing_share = (
    df[key_columns]
    .isna()
    .mean()
    .rename("missing_ratio")
    .sort_values(ascending=False)
)
display(missing_share.to_frame())


,missing_ratio
AwardAmount,0.90
ResponseWindowDays,0.49
SetASide,0.00
NaicsCode,0.00
ClassificationCode,0.00


## Dinamiche pluriennali
Analizziamo come evolvono volumi, valori economici e tempi di risposta nel tempo.


In [5]:
yearly_stats = yearly_summary(df)
yearly_stats["total_awarded_mln"] = yearly_stats["total_awarded"] / 1e6
yearly_stats["opportunity_growth_pct"] = yearly_stats["total_opportunities"].pct_change() * 100
yearly_stats["award_growth_pct"] = yearly_stats["total_awarded"].pct_change() * 100
display(yearly_stats)


,FiscalYear,total_opportunities,total_awarded,median_award,avg_award,median_response_window_days,total_awarded_mln,opportunity_growth_pct,award_growth_pct
0,2004,227,"569,589,085.18","1,960,790.80","35,599,317.82",15.00,569.59,NaN,NaN
1,2005,375,"250,000,306,619,944.34","250,000.00","9,259,270,615,553.49",14.00,"250,000,306.62",65.20,"43,891,244.33"
2,2006,635,"540,763,168.15","813,848.00","11,505,599.32",14.00,540.76,69.33,-100.00
3,2007,646,"1,134,495,940.04","2,417,861.25","24,138,211.49",13.00,"1,134.50",1.73,109.80
4,2008,717,"10,000,008,070,652,248.00","3,103,775.76","175,438,738,081,618.38",14.00,"10,000,008,070.65",10.99,"881,449,336.51"
5,2009,621,"10,000,017,442,549,380.00","4,034,698.69","138,889,131,146,519.17",13.00,"10,000,017,442.55",-13.39,0.00
6,2010,859,"4,697,373,748,296,810,682,474,531,949,602,385,4...","927,670.34","63,478,023,625,632,580,108,616,068,362,485,628,...",13.00,"4,697,373,748,296,810,820,271,710,180,212,736.00",38.33,"46,973,655,548,937,462,144,303,104.00"
7,2011,995,"9,679,632,242.68","5,870,105.99","83,445,105.54",13.00,"9,679.63",15.83,-100.00
8,2012,1065,"5,502,856,230.47","5,631,807.89","50,952,372.50",12.00,"5,502.86",7.04,-43.15
9,2013,970,"14,972,383,971.10","7,615,420.63","174,097,488.04",14.00,"14,972.38",-8.92,172.08


In [6]:
fig_volume = px.bar(
    yearly_stats,
    x="FiscalYear",
    y="total_opportunities",
    title="Volume di opportunità per anno fiscale",
    labels={"FiscalYear": "Anno fiscale", "total_opportunities": "Numero opportunità"},
    text_auto=True,
    template="plotly_white",
)
fig_award = px.line(
    yearly_stats,
    x="FiscalYear",
    y="total_awarded_mln",
    markers=True,
    title="Valore totale aggiudicato (milioni $)",
    labels={"FiscalYear": "Anno fiscale", "total_awarded_mln": "Milioni di $"},
    template="plotly_white",
)
fig_response = px.line(
    yearly_stats,
    x="FiscalYear",
    y="median_response_window_days",
    markers=True,
    title="Tempo mediano di risposta (giorni)",
    labels={"FiscalYear": "Anno fiscale", "median_response_window_days": "Giorni"},
    template="plotly_white",
)
fig_growth = px.bar(
    yearly_stats,
    x="FiscalYear",
    y=["opportunity_growth_pct", "award_growth_pct"],
    title="Tassi di crescita YoY",
    labels={"value": "Crescita %", "variable": "Indicatore"},
    barmode="group",
    template="plotly_white",
)

fig_volume.show()
fig_award.show()
fig_response.show()
fig_growth.show()


## Mix istituzionale
Identifichiamo le agenzie e i sub-tier più attivi sia per numero di opportunità sia per valore economico.


In [7]:
top_departments = agency_mix(df, column="Department/Ind.Agency", top_n=15)
fig_dept = px.bar(
    top_departments,
    x="opportunities",
    y="entity",
    orientation="h",
    title="Top dipartimenti per numero di opportunità",
    text_auto=True,
    labels={"opportunities": "Opportunità", "entity": "Dipartimento"},
    template="plotly_white",
)
fig_dept_award = px.bar(
    top_departments.sort_values("total_award", ascending=True),
    x="total_award",
    y="entity",
    orientation="h",
    title="Top dipartimenti per valore economico",
    labels={"total_award": "Valore totale ($)", "entity": "Dipartimento"},
    template="plotly_white",
)
fig_dept.show()
fig_dept_award.show()

subtier = agency_mix(df, column="Sub-Tier", top_n=15)
fig_subtier = px.bar(
    subtier,
    x="opportunities",
    y="entity",
    orientation="h",
    title="Top sub-tier per numero di opportunità",
    text_auto=True,
    labels={"opportunities": "Opportunità", "entity": "Sub-Tier"},
    template="plotly_white",
)
fig_subtier.show()


## Programmi di Set Aside
Valutiamo la composizione dei principali programmi di set aside sia in termini di volumi che di valore.


In [8]:
setaside = set_aside_landscape(df, top_n=12)
fig_setaside_bar = px.bar(
    setaside,
    x="share",
    y="SetASide",
    orientation="h",
    title="Composizione percentuale dei principali Set Aside",
    text_auto=".1%",
    labels={"share": "Quota", "SetASide": "Programma"},
    template="plotly_white",
)
fig_setaside_value = px.bar(
    setaside.sort_values("total_award", ascending=True),
    x="total_award",
    y="SetASide",
    orientation="h",
    title="Valore economico per Set Aside",
    labels={"total_award": "Valore totale ($)", "SetASide": "Programma"},
    template="plotly_white",
)
fig_setaside_bar.show()
fig_setaside_value.show()


## Leader economici (Top Awardees)
Analizziamo i principali assegnatari per volume economico e frequenza di aggiudicazioni.

In [9]:
awardees = awardee_leaderboard(df, top_n=20)
awardees["total_award_mln"] = awardees["total_award"] / 1e6
display(awardees)

fig_awardees_value = px.bar(
    awardees.sort_values("total_award", ascending=True),
    x="total_award_mln",
    y="Awardee",
    orientation="h",
    title="Top awardee per valore aggiudicato (milioni $)",
    labels={"total_award_mln": "Valore totale (milioni $)", "Awardee": "Awardee"},
    template="plotly_white",
)
fig_awardees_count = px.bar(
    awardees.sort_values("awards_count", ascending=True),
    x="awards_count",
    y="Awardee",
    orientation="h",
    title="Top awardee per numero di aggiudicazioni",
    labels={"awards_count": "Numero aggiudicazioni", "Awardee": "Awardee"},
    template="plotly_white",
)
fig_awardees_value.show()
fig_awardees_count.show()


,Awardee,total_award,avg_award,median_award,awards_count,opportunities,total_award_mln
0,"American Security Programs, Inc.; Diversified ...","716,421,500,649,829,615,769,257,556,344,593,948...","716,421,500,649,829,615,769,257,556,344,593,948...","716,421,500,649,829,615,769,257,556,344,593,948...",1,1,"716,421,500,649,829,674,770,504,393,317,941,248.00"
1,"American Security Programs, Coastal Internatio...","4,697,373,748,296,810,682,474,531,949,602,385,4...","4,697,373,748,296,810,682,474,531,949,602,385,4...","4,697,373,748,296,810,682,474,531,949,602,385,4...",1,1,"4,697,373,748,296,810,820,271,710,180,212,736.00"
2,"Hawk One Security, Inc. 1400 I Street, Inc. Su...","10,000,005,353,000,000.00","10,000,005,353,000,000.00","10,000,005,353,000,000.00",1,1,"10,000,005,353.00"
3,"Systems Integration/Modeling Simulation, Inc. ...","10,000,005,353,000,000.00","10,000,005,353,000,000.00","10,000,005,353,000,000.00",1,1,"10,000,005,353.00"
4,"Applied Security Technologies, 20715 Durand Oa...","250,000,250,000,000.00","250,000,250,000,000.00","250,000,250,000,000.00",1,1,"250,000,250.00"
5,"G4S Tri-parte 1299 Farnam Street, Suite 1300 O...","43,317,371,124.23","21,658,685,562.12","21,658,685,562.12",2,2,"43,317.37"
6,G4S Secure Solutions 1395 University Blvd. JUP...,"29,472,194,092.76","29,472,194,092.76","29,472,194,092.76",1,1,"29,472.19"
7,CONTINUITY GLOBAL SOLUTIONS LLC Port Saint Joe...,"20,684,841,348.09","5,171,210,337.02","5,171,765,989.62",4,4,"20,684.84"
8,"UNIVERSAL PROTECTION SERVICE, LIMITED PARTNERS...","13,808,436,840.45","4,602,812,280.15","3,299,325,000.00",3,3,"13,808.44"
9,TRIPLE CANOPY INC Herndon VA 20171 USA,"13,438,765,190.45","6,719,382,595.23","6,719,382,595.23",2,2,"13,438.77"


### Concentrazione del valore aggiudicato
Valutiamo quanto il budget si concentra su pochi operatori.

In [10]:
concentration = award_concentration(df, top_k=50)
concentration["rank"] = range(1, len(concentration) + 1)
concentration["cumulative_share_pct"] = concentration["cumulative_share"] * 100

display(concentration.head(20))

top10_share = concentration.head(10)["share"].sum() * 100
print(f"Quota top 10 awardee: {top10_share:.1f}% del valore complessivo")

top20_share = concentration.head(20)["share"].sum() * 100
print(f"Quota top 20 awardee: {top20_share:.1f}% del valore complessivo")

fig_concentration = px.line(
    concentration,
    x="rank",
    y="cumulative_share_pct",
    markers=True,
    title="Curva di concentrazione del valore aggiudicato",
    labels={"rank": "Rank awardee", "cumulative_share_pct": "Quota cumulata %"},
    template="plotly_white",
)
fig_concentration.update_yaxes(range=[0, 105])
fig_concentration.show()


,Awardee,total_award,avg_award,median_award,awards_count,opportunities,share,cumulative_share,rank,cumulative_share_pct
0,"American Security Programs, Inc.; Diversified ...","716,421,500,649,829,615,769,257,556,344,593,948...","716,421,500,649,829,615,769,257,556,344,593,948...","716,421,500,649,829,615,769,257,556,344,593,948...",1,1,0.99,0.99,1,99.35
1,"American Security Programs, Coastal Internatio...","4,697,373,748,296,810,682,474,531,949,602,385,4...","4,697,373,748,296,810,682,474,531,949,602,385,4...","4,697,373,748,296,810,682,474,531,949,602,385,4...",1,1,0.01,1.00,2,100.00
2,"Hawk One Security, Inc. 1400 I Street, Inc. Su...","10,000,005,353,000,000.00","10,000,005,353,000,000.00","10,000,005,353,000,000.00",1,1,0.00,1.00,3,100.00
3,"Systems Integration/Modeling Simulation, Inc. ...","10,000,005,353,000,000.00","10,000,005,353,000,000.00","10,000,005,353,000,000.00",1,1,0.00,1.00,4,100.00
4,"Applied Security Technologies, 20715 Durand Oa...","250,000,250,000,000.00","250,000,250,000,000.00","250,000,250,000,000.00",1,1,0.00,1.00,5,100.00
5,"G4S Tri-parte 1299 Farnam Street, Suite 1300 O...","43,317,371,124.23","21,658,685,562.12","21,658,685,562.12",2,2,0.00,1.00,6,100.00
6,G4S Secure Solutions 1395 University Blvd. JUP...,"29,472,194,092.76","29,472,194,092.76","29,472,194,092.76",1,1,0.00,1.00,7,100.00
7,CONTINUITY GLOBAL SOLUTIONS LLC Port Saint Joe...,"20,684,841,348.09","5,171,210,337.02","5,171,765,989.62",4,4,0.00,1.00,8,100.00
8,"UNIVERSAL PROTECTION SERVICE, LIMITED PARTNERS...","13,808,436,840.45","4,602,812,280.15","3,299,325,000.00",3,3,0.00,1.00,9,100.00
9,TRIPLE CANOPY INC Herndon VA 20171 USA,"13,438,765,190.45","6,719,382,595.23","6,719,382,595.23",2,2,0.00,1.00,10,100.00


Quota top 10 awardee: 100.0% del valore complessivo
Quota top 20 awardee: 100.0% del valore complessivo


### Distribuzione statistica degli importi
Esploriamo la struttura dei valori di aggiudicazione per cogliere bandi outlier.

In [11]:
award_stats = award_amount_summary(df)
display(award_stats)

p99 = df["AwardAmount"].quantile(0.99)
print(f"99° percentile: ${p99:,.0f}")

fig_award_hist = px.histogram(
    df[df["AwardAmount"].notna()],
    x="AwardAmount",
    nbins=60,
    title="Distribuzione degli importi aggiudicati (fino al 99° percentile)",
    labels={"AwardAmount": "Valore aggiudicato ($)"},
    template="plotly_white",
)
fig_award_hist.update_xaxes(tickprefix="$", range=[0, p99])
fig_award_hist.update_yaxes(title_text="Frequenza")
fig_award_hist.show()

fig_award_box = px.box(
    df[df["AwardAmount"].notna()],
    x="AwardAmount",
    points="suspectedoutliers",
    title="Boxplot degli importi aggiudicati",
    labels={"AwardAmount": "Valore aggiudicato ($)"},
    template="plotly_white",
)
fig_award_box.update_xaxes(tickprefix="$")
fig_award_box.show()


,AwardAmount
count,"1,728.00"
mean,"417,314,163,424,841,648,742,914,153,124,601,004..."
std,"17,234,727,784,416,695,886,414,670,500,485,962,..."
min,0.00
10%,"50,000.00"
25%,"250,000.00"
50%,"3,722,365.75"
75%,"31,789,343.00"
90%,"130,822,011.90"
max,"716,421,500,649,829,615,769,257,556,344,593,948..."


99° percentile: $9,115,118,335


## Settori NAICS ad alto potenziale
Costruiamo una matrice anno-settore per capire quali codici NAICS guidano la domanda nel tempo.


In [12]:
naics_matrix = naics_opportunity_matrix(df, top_n=12)
fig_naics = px.density_heatmap(
    naics_matrix,
    x="FiscalYear",
    y="NaicsCode",
    z="opportunities",
    color_continuous_scale="Blues",
    title="Heatmap opportunità per codice NAICS (top 12)",
    labels={"opportunities": "Numero opportunità"},
)
fig_naics_value = px.density_heatmap(
    naics_matrix,
    x="FiscalYear",
    y="NaicsCode",
    z="total_award",
    color_continuous_scale="Greens",
    title="Heatmap valore aggiudicato per codice NAICS (top 12)",
    labels={"total_award": "Valore ($)"},
)
fig_naics.show()
fig_naics_value.show()


## Distribuzione geografica
Analizziamo la concentrazione geografica delle opportunità e dei budget associati.


In [13]:
state_geo = geographic_distribution(df, level="State").head(20)
fig_state = px.bar(
    state_geo.sort_values("opportunities", ascending=True),
    x="opportunities",
    y="State",
    orientation="h",
    title="Top stati per volume di opportunità",
    text_auto=True,
    labels={"opportunities": "Opportunità", "State": "Stato"},
    template="plotly_white",
)
fig_state_value = px.bar(
    state_geo.sort_values("total_award", ascending=True),
    x="total_award",
    y="State",
    orientation="h",
    title="Top stati per valore economico",
    labels={"total_award": "Valore totale ($)", "State": "Stato"},
    template="plotly_white",
)
fig_state.show()
fig_state_value.show()

country_geo = geographic_distribution(df, level="PopCountry")
fig_country = px.bar(
    country_geo.head(20),
    x="PopCountry",
    y="opportunities",
    title="Distribuzione internazionale delle opportunità (sede di performance)",
    labels={"PopCountry": "Paese", "opportunities": "Opportunità"},
    template="plotly_white",
)
fig_country.show()


## Cadenzamento temporale
Esploriamo la pipeline trimestrale per evidenziare stagionalità e finestre di picco.


In [14]:
timeline = timeline_by_quarter(df)
fig_timeline = px.line(
    timeline,
    x="Quarter",
    y="opportunities",
    title="Opportunità pubblicate per trimestre",
    labels={"Quarter": "Trimestre", "opportunities": "Numero opportunità"},
    markers=True,
    template="plotly_white",
)
fig_timeline_award = px.line(
    timeline,
    x="Quarter",
    y="total_award",
    title="Valore economico pubblicato per trimestre",
    labels={"Quarter": "Trimestre", "total_award": "Valore ($)"},
    markers=True,
    template="plotly_white",
)
fig_timeline_response = px.line(
    timeline.dropna(subset=["median_response_days"]),
    x="Quarter",
    y="median_response_days",
    title="Durata mediana finestra di risposta per trimestre",
    labels={"Quarter": "Trimestre", "median_response_days": "Giorni"},
    markers=True,
    template="plotly_white",
)
fig_timeline.show()
fig_timeline_award.show()
fig_timeline_response.show()


## Profondità pipeline
Analizziamo la distribuzione della finestra di risposta per identificare bandi rapidi vs
opportunità più strutturate.


In [15]:
response_profile = opportunity_duration_profile(df)
fig_response_box = px.box(
    response_profile,
    x="FiscalYear",
    y="ResponseWindowDays",
    title="Distribuzione tempi di risposta per anno fiscale",
    labels={"FiscalYear": "Anno fiscale", "ResponseWindowDays": "Giorni"},
    template="plotly_white",
)
fig_response_box.show()


## Insight per il management
- Crescita delle opportunità: confrontando volumi e valore economico emerge dove si concentrano i trend positivi o
  i rallentamenti, utili per dimensionare la forza commerciale.
- Agenzie chiave: il mix dipartimentale e sub-tier consente di orientare account plan dedicati e partnership locali.
- Programmi di Set Aside: l'analisi delle quote e dei valori guida politiche di certificazione e partecipazione
  per PMI, imprese femminili e veterani.
- Settori NAICS: la heatmap identifica cluster settoriali in espansione dove concentrare l'offerta.
- Geografia e stagionalità: le analisi spaziali e temporali supportano piani di delivery, staffing e cash-flow.
- Tempi di risposta: la distribuzione della finestra media orienta il capacity planning del team di proposal.

Prossimi passi suggeriti: collegare l'analisi a metriche interne (win rate, margini) e integrare dati FPDS/USAspending
per completare la visione sul ciclo di vita dei contratti.
